# Create validation set

In [1]:
path = 'data/fish'
trainpath = path + '/train'
validpath = path + '/valid'
models_path = path+'/models/'

In [4]:
import os
import random

if not os.path.exists(models_path):
    os.makedirs(models_path)

valid_perc = 0.15
for fish_type in os.listdir(trainpath):
    type_train_path = os.path.join(trainpath, fish_type)
    type_valid_path = os.path.join(validpath, fish_type)
    if not os.path.exists(type_valid_path):
        os.makedirs(type_valid_path)
        images = os.listdir(type_train_path)
        valid_images = random.sample(images, int(len(images) * valid_perc))
        for image in valid_images:
            os.rename(os.path.join(type_train_path, image), os.path.join(type_valid_path, image))

# Build the VGG16 Model

In [21]:
from keras import backend as K
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers.core import Lambda, Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D
import numpy as np

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3, 1, 1))
def vgg_preprocess(x):
    # subtracts the mean so that we get a 0-centered value
    x = x - vgg_mean
    
    # reverses the axis, since most pretrained data comes from OpenCV, which uses BGR rather than RGB
    return x[:, ::-1]

def vgg_convblock(model, layers, filters):
    for _ in xrange(0, layers):
        model.add(Convolution2D(filters, 3, 3, border_mode='same', activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))

model = Sequential()
model.add(Lambda(vgg_preprocess, input_shape=(3, 224, 224), output_shape=(3,224,224)))
vgg_convblock(model, 2, 64)
vgg_convblock(model, 2, 128)
vgg_convblock(model, 3, 256)
vgg_convblock(model, 3, 512)
vgg_convblock(model, 3, 512)

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax'))

download_path = 'http://files.fast.ai/models/'
weights_file = get_file('vgg16.h5', download_path+'vgg16.h5', cache_subdir='models')
model.load_weights(weights_file)

In [22]:
conv_layers = [layer for layer in model.layers if type(layer) in [Convolution2D, MaxPooling2D]]
for layer in conv_layers:
    layer.trainable = False

In [23]:
from keras.layers.normalization import BatchNormalization
model = Sequential([BatchNormalization(axis=1, input_shape=(3, 224, 224))] + conv_layers + [
    BatchNormalization(axis=1),
    Flatten(),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dense(4096, activation='relu'),
    BatchNormalization(),
    Dense(8, activation='softmax')
])
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_14 (BatchNorm (None, 3, 224, 224)   12          batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_28 (Convolution2D) (None, 64, 224, 224)  1792        lambda_3[0][0]                   
                                                                   batchnormalization_14[0][0]      
____________________________________________________________________________________________________
convolution2d_29 (Convolution2D) (None, 64, 224, 224)  36928       convolution2d_28[0][0]           
                                                                   convolution2d_28[1][0]           
___________________________________________________________________________________________

In [28]:
from keras.optimizers import Adam
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [29]:
from keras.preprocessing.image import ImageDataGenerator
def batch_gen(path, class_mode='categorical', shuffle=True):
    return ImageDataGenerator().flow_from_directory(path, target_size=(224, 224), batch_size=16, class_mode=class_mode, shuffle=shuffle)

In [30]:
train_gen = batch_gen(trainpath)
valid_gen = batch_gen(validpath)

Found 3214 images belonging to 8 classes.
Found 563 images belonging to 8 classes.


In [31]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 110s - loss: 5.0484 - acc: 0.5765 - val_loss: 2.8254 - val_acc: 0.6909
Epoch 2/3
3214/3214 [==============================] - 107s - loss: 1.7700 - acc: 0.8037 - val_loss: 0.6561 - val_acc: 0.9130
Epoch 3/3
3214/3214 [==============================] - 108s - loss: 0.6295 - acc: 0.8995 - val_loss: 0.6521 - val_acc: 0.8863


In [32]:
model.save_weights(models_path+'first_cut.h5')

In [33]:
model.optimizer.lr = 0.0001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 107s - loss: 0.3538 - acc: 0.9384 - val_loss: 0.3427 - val_acc: 0.9432
Epoch 2/3
3214/3214 [==============================] - 107s - loss: 0.1696 - acc: 0.9639 - val_loss: 0.3189 - val_acc: 0.9449
Epoch 3/3
3214/3214 [==============================] - 107s - loss: 0.2159 - acc: 0.9627 - val_loss: 0.3623 - val_acc: 0.9449


In [34]:
model.save_weights(models_path+'second_cut.h5')

In [35]:
model.optimizer.lr = 0.00001
model.fit_generator(train_gen, samples_per_epoch=train_gen.nb_sample, nb_epoch=3, 
                    validation_data=valid_gen, nb_val_samples=valid_gen.nb_sample)

Epoch 1/3
3214/3214 [==============================] - 116s - loss: 0.2514 - acc: 0.9571 - val_loss: 0.3904 - val_acc: 0.9520
Epoch 2/3
3214/3214 [==============================] - 108s - loss: 0.2511 - acc: 0.9586 - val_loss: 0.3589 - val_acc: 0.9414
Epoch 3/3
3214/3214 [==============================] - 108s - loss: 0.1957 - acc: 0.9689 - val_loss: 0.7567 - val_acc: 0.9076


In [36]:
model.load_weights(models_path+'second_cut.h5')

# Generate a test submission for part 1

In [58]:
testpath = path + '/test_stg1'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 1000 images belonging to 1 classes.


In [59]:
predictions_one = model.predict_generator(test_gen, test_gen.nb_sample)

In [60]:
import pandas as pd

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_one = pd.DataFrame(predictions_one, columns=classes)
submission_one.insert(0, 'image', [os.path.basename(filename) for filename in test_gen.filenames])
submission_one.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,2.075544e-17,7.957907e-21,2.798510e-22,5.780269e-18,1.000000e+00,5.006262e-33,3.334641e-22,4.182636e-24
1,img_00007.jpg,9.999990e-01,4.539869e-08,2.045214e-08,1.361729e-10,8.662252e-11,5.680070e-07,2.994132e-07,1.238451e-09
2,img_00009.jpg,9.999945e-01,2.478718e-06,1.911671e-08,5.920632e-10,3.989150e-10,9.096841e-07,2.278376e-07,1.884298e-06
3,img_00018.jpg,9.999998e-01,1.255315e-10,2.862696e-10,6.951282e-12,1.090699e-11,6.353689e-10,3.931809e-09,2.154998e-07
4,img_00027.jpg,9.713554e-01,4.695924e-08,6.890896e-09,3.790317e-09,1.291641e-08,3.784194e-11,1.630878e-05,2.862824e-02


In [61]:
# submission_one.to_csv(os.path.join(path, 'stage_one.csv.gz'), index=False, compression='gzip')

# Generate a test submission for part 2

In [62]:
testpath = path + '/test_stg2'
test_gen = batch_gen(testpath, class_mode=None, shuffle=False)

Found 12153 images belonging to 1 classes.


In [63]:
predictions_two = model.predict_generator(test_gen, test_gen.nb_sample)

In [64]:
import pandas as pd

classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission_two = pd.DataFrame(predictions_two, columns=classes)
submission_two.insert(0, 'image', ['test_stg2/' + os.path.basename(filename) for filename in test_gen.filenames])
submission_two.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,test_stg2/image_00001.jpg,0.833023,0.001580,0.000535,0.002036,0.001063,0.125616,2.045767e-03,0.034101
1,test_stg2/image_00002.jpg,0.926447,0.006298,0.001134,0.006286,0.000659,0.030217,2.388213e-03,0.026570
2,test_stg2/image_00003.jpg,0.997816,0.000001,0.000005,0.000016,0.002148,0.000004,8.701380e-09,0.000011
3,test_stg2/image_00004.jpg,0.964620,0.016447,0.000027,0.000186,0.000210,0.002062,3.778702e-06,0.016445
4,test_stg2/image_00005.jpg,0.939952,0.025057,0.000103,0.000521,0.001351,0.000328,1.474283e-06,0.032686


In [65]:
submission = submission_one.append(submission_two, ignore_index=True)
submission.head()

,image,ALB,BET,DOL,LAG,NoF,OTHER,SHARK,YFT
0,img_00005.jpg,2.075544e-17,7.957907e-21,2.798510e-22,5.780269e-18,1.000000e+00,5.006262e-33,3.334641e-22,4.182636e-24
1,img_00007.jpg,9.999990e-01,4.539869e-08,2.045214e-08,1.361729e-10,8.662252e-11,5.680070e-07,2.994132e-07,1.238451e-09
2,img_00009.jpg,9.999945e-01,2.478718e-06,1.911671e-08,5.920632e-10,3.989150e-10,9.096841e-07,2.278376e-07,1.884298e-06
3,img_00018.jpg,9.999998e-01,1.255315e-10,2.862696e-10,6.951282e-12,1.090699e-11,6.353689e-10,3.931809e-09,2.154998e-07
4,img_00027.jpg,9.713554e-01,4.695924e-08,6.890896e-09,3.790317e-09,1.291641e-08,3.784194e-11,1.630878e-05,2.862824e-02


In [66]:
submission.to_csv(os.path.join(path, 'stage_two.csv.gz'), index=False, compression='gzip')